# Customer segmentation
Data set from kaggle: https://www.kaggle.com/code/fabiendaniel/customer-segmentation/

In [ ]:
%%capture
%pip install graphdatascience pandas ipython numpy

In [ ]:
import pandas as pd
from neo4j import GraphDatabase
from graphdatascience import GraphDataScience
import numpy as np

In [ ]:
# Neo4j Connection details
DB_ULR = 'neo4j://localhost:7687'
DB_USER = 'neo4j'
DB_PASS = 'test1234'
DB_NAME = 'custseg'
gds = GraphDataScience.from_neo4j_driver(DB_ULR, auth=(DB_USER, DB_PASS))
gds.version()

In [ ]:
gds.run_cypher("create database {dbname} if not exists wait".format(dbname = DB_NAME), database="system")

In [ ]:
gds.set_database(DB_NAME)

In [ ]:
# Read the csv file with pandas
raw_csv = pd.read_csv('./datasets/data.csv',encoding="ISO-8859-1",
                         dtype={'CustomerID': str,'InvoiceID': str})
raw_csv.head()

In [ ]:
# Select customers
df_customers = raw_csv[['CustomerID','Country']].drop_duplicates().dropna()
df_customers.head()

In [ ]:
# Create Customer nodes
gds.run_cypher('create constraint if not exists for (n:Customer) require (n.id) is node key')
create_customer_res = gds.run_cypher('''
    unwind $data as row
    merge (n:Customer{id: row.CustomerID})
        set n.country = row.Country
    return count(*) as custmers_created
''', params = {'data': df_customers.to_dict('records')})
create_customer_res.head()

In [ ]:
# Select products
df_products = raw_csv[['StockCode','Description']].drop_duplicates().dropna()
df_products.head()

In [ ]:
# Create Product nodes
gds.run_cypher('create constraint if not exists for (n:Product) require (n.id) is node key')
create_product_res = gds.run_cypher('''
    unwind $data as row
    merge (n:Product{id: row.StockCode})
        set n.description = row.Description
    return count(*) as products_created
''', params = {'data': df_products.to_dict('records')})
create_product_res.head()

In [ ]:
# Select invoice data
df_invoices = raw_csv[['InvoiceNo', 'InvoiceDate']].drop_duplicates().dropna()
df_invoices.head()

In [ ]:
# Create Invoice nodes
gds.run_cypher('create constraint if not exists for (n:Invoice) require (n.id) is node key')
create_invoice_res = gds.run_cypher('''
    unwind $data as row
    with row,  apoc.date.parse(row.InvoiceDate, 'ms', 'dd/MM/yyyy HH:mm') as ms
    merge (n:Invoice{id: row.InvoiceNo})
        set n.invoice_date = datetime( { epochmillis: ms } )
    return count(*) as invoices_created
''', params = {'data': df_invoices.to_dict('records')})
create_invoice_res.head()

In [ ]:
# Select data for cust - invoice relationship
df_billed_to = raw_csv[['CustomerID','InvoiceNo']].drop_duplicates().dropna()
df_billed_to.head()

In [ ]:
# Create billed_to relationship
create_bill_to_res = gds.run_cypher('''
    unwind $data as row
    match (i:Invoice{id: row.InvoiceNo}), (c:Customer{id: row.CustomerID})
    merge (i)-[:billed_to]->(c)
    return count(*) as bill_to_rels_created
''', params = {'data': df_billed_to.to_dict('records')})
create_bill_to_res.head()

In [ ]:
# Select data for invoice - product relationship
df_line_item = raw_csv[['InvoiceNo','StockCode','Quantity','UnitPrice']].drop_duplicates().dropna()

# We want to store prices in cents so we can represent them as 
# integer values to avoid rounding errors later on
df_line_item['UnitPrice'] = df_line_item['UnitPrice'] * 100 

df_line_item.head()

In [ ]:
# Create line_item relationship
create_line_item_res = gds.run_cypher('''
    unwind $data as row
    match (i:Invoice{id: row.InvoiceNo}), (p:Product{id: row.StockCode})
    merge (p)-[li:line_item]->(i)
        set li.qty = toInteger(row.Quantity),
            li.price = toInteger(row.UnitPrice)
    return count(*) as line_item_rels_created
''', params = {'data': df_line_item.to_dict('records')})
create_line_item_res.head()

## Graph model so far
![](./images/graph_model.png)

## Basic data exploration
Let's just get some picture of our graph

In [ ]:
# Flag cancelled orders (invoice number starts with C)
cancellations = gds.run_cypher('''
    call {
        // reset just in case we change the logic/rerun
        match (i:CancelledOrder)
        set i:Invoice remove i:CancelledOrder
    }
    match (i:Invoice)
    where i.id starts with 'C'
    set i:CancelledOrder remove i:Invoice
    return count(*) as number_of_cancelled_orders
''')
cancellations.head()

### Basic product analysis
Why start with products? If we don't understand the products we have no domain knowledge when we start looking at customers

In [ ]:
# Product sales
df_customerinvoices = gds.run_cypher('''
    match (p:Product)
    return p.id as productId, count { (p)-[:line_item]->(:Invoice) } as number_of_times_ordered
''')
df_customerinvoices.hist('number_of_times_ordered')

**note**: We should look out for product that are only ordered once or twice (may not be of interest), we shoul also investigate products that are ordereed a lot (might be an invoice fee or something else that stands out)

In [ ]:
# First sold, last sold dates (yea we are going to assume some products are no longer sold, it may be 
# relevant to know both for customer segmentation but also for recommendations)
gds.run_cypher('''
    match (p:Product)-[:line_item]->(i:Invoice)
    with p, min(i.invoice_date) as first_invoice_date, max(i.invoice_date) as last_invoice_date
    set p.first_sold = first_invoice_date,
        p.last_sold = last_invoice_date  
''')

In [ ]:
# Let's just assume that products that have not been invoiced the past 5 quarters are no longer sold
# Why 5? Some products may be seasonal
discontiued_products = gds.run_cypher('''
    call { 
        // reset just in case we change the cutoff
        match (d:DiscontinuedProduct)
        set d:Product remove d:DiscontinuedProduct 
    }
    match (p:Product)
    with max(p.last_sold)  - duration({days:365+90})  as for_sale_cutoff_date
    match (p:Product) where p.last_sold < for_sale_cutoff_date
        set p:DiscontinuedProduct remove p:Product
    return count(*) as number_of_discontiued_products 
''')
discontiued_products.head()

In [ ]:
# Top products (still in sales)
gds.run_cypher('''
    match (p:Product)
    return p.id as productId, 
        p.description as description, 
        count { (p)-[:line_item]->() } as number_of_times_ordered 
    order by number_of_times_ordered desc limit 10
''').head(10)

In [ ]:
# Product "damaged" stands out, what others are there
gds.run_cypher('''
    match (p:Product)-[li:line_item]->()
    with p, count(distinct(li.price)) as numberOfPrices
    where numberOfPrices>1
    match  (p)-[li:line_item]->()
    return p.id as productId, 
        collect(li.price) as prices,
        p.description as description 
    order by productId desc limit 25
''').head(25)

In [ ]:
# Let's treat a few of these as misc fees
misc_products = gds.run_cypher('''
    call { 
        // reset just in case we the logic
        match (d:AdminCharge)
        set d:Product remove d:AdminCharge 
    }
    match (p:Product)
    where p.id in $admin_products
        set p:AdminCharge remove p:Product
    return count(*) as number_of_admin_charges
''', params = { 'admin_products': ['POST', 'D', 'C2', 'M', 'BANK CHARGES', 'PADS', 'DOT'] })
misc_products.head()

## Basic analysis of customers

In [ ]:
# Check number of invoices per customer
df_customerinvoices = gds.run_cypher('''
    match (c:Customer)
    return c.id as custId, count { (c)<-[:billed_to]-() } as number_of_invoices
''')
df_customerinvoices.hist('number_of_invoices')

**note**: We may want to segment customer based on number of invoices, like B2B vs B2C

In [ ]:
# Let's set first and last invoice date on customer nodes
gds.run_cypher('''
    match (c:Customer)<-[:billed_to]-(i:Invoice)
    with c, min(i.invoice_date) as first_invoice_date, max(i.invoice_date) as last_invoice_date
    set c.first_invoice_date = first_invoice_date,
        c.last_invoice_date = last_invoice_date  
''')

In [ ]:
# What is the max last_invoice_date
max_last_invoice = gds.run_cypher('''
    match (c:Customer)
    return max(c.last_invoice_date) as max_last_invoice_date
''')
max_last_invoice.head()

In [ ]:
# Let's just assume that customers that have not been invoiced the past quarter are churned 
curned_customers = gds.run_cypher('''
    call {
        // reset just in case we change the cutoff
        match (c:ChurnedCustomer)
        set c:Customer remove c:ChurnedCustomer
    }
    match (c:Customer)
    with max(c.last_invoice_date)  - duration({days:90})  as churn_cutoff_date
    match (c:Customer) where c.last_invoice_date < churn_cutoff_date
        set c:ChurnedCustomer remove c:Customer
    return count(*) as number_of_churned_customers 
''')
curned_customers.head()

In [ ]:
# Customers by revenue (also includes admin charges)
df_customer_order_value = gds.run_cypher('''
    match (c:Customer)<-[:billed_to]-(:Invoice)<-[li:line_item]-()
    return c.id as customerId,
       sum(li.qty * li.price) as total_order_value

''')
df_customer_order_value.hist('total_order_value')

In [ ]:
# Hmm, scary close to 0, do we also have customers with a total amount that is negative
data = gds.run_cypher('''
    match (c:Customer)<-[:billed_to]-(:Invoice)<-[li:line_item]-(p)
    with c.id as customerId, sum(li.qty * li.price) as total_order_value
    where total_order_value<0
    with collect(customerId) as custIds
    match (c:Customer)<-[:billed_to]-()<-[li:line_item]-(p:Product)
    where c.id in custIds
    return c.id as customerId, li.qty as qty, li.price as price, p.id as productId, p.description as product, li.qty*li.price as value
    limit 100
''')
data.head(100)
# Remark: It is important that we only go through Invoice nodes as we also have Cancelled invoices (assuming they were never sent/paid)
